In [ ]:
pip install pandas scikit-learn flask


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from flask import Flask, request, jsonify
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/dataset/human+activity+recognition+using+smartphones/UCI HAR Dataset'

In [ ]:




# Load Activity Labels
def load_activity_labels(dataset_path):
    activity_labels_path = f"{dataset_path}/activity_labels.txt"
    if not os.path.exists(activity_labels_path):
        raise FileNotFoundError(f"File not found: {activity_labels_path}")
    activity_labels = pd.read_csv(activity_labels_path, header=None, delim_whitespace=True, names=["Activity", "Label"])
    return dict(zip(activity_labels["Activity"], activity_labels["Label"]))

# Load Data
def load_data(dataset_path):
    # Paths
    train_path = f"{dataset_path}/train"
    test_path = f"{dataset_path}/test"
    features_path = f"{dataset_path}/features.txt"

    # Check if files exist
    for path in [features_path, train_path, test_path]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"File or directory not found: {path}")

    activity_labels = load_activity_labels(dataset_path)

    # Load Features
    features = pd.read_csv(features_path, header=None, delim_whitespace=True, names=["Index", "FeatureName"])
    feature_names = features["FeatureName"].tolist()

    # Load Training Data
    subject_train = pd.read_csv(f"{train_path}/subject_train.txt", header=None, names=["Subject"])
    y_train = pd.read_csv(f"{train_path}/y_train.txt", header=None, names=["Activity"])
    X_train = pd.read_csv(f"{train_path}/X_train.txt", header=None, delim_whitespace=True)
    X_train.columns = feature_names
    y_train["Activity"] = y_train["Activity"].map(activity_labels)

    # Load Test Data
    subject_test = pd.read_csv(f"{test_path}/subject_test.txt", header=None, names=["Subject"])
    y_test = pd.read_csv(f"{test_path}/y_test.txt", header=None, names=["Activity"])
    X_test = pd.read_csv(f"{test_path}/X_test.txt", header=None, delim_whitespace=True)
    X_test.columns = feature_names
    y_test["Activity"] = y_test["Activity"].map(activity_labels)

    # Combine Data
    train_data = pd.concat([subject_train, y_train, X_train], axis=1)
    test_data = pd.concat([subject_test, y_test, X_test], axis=1)

    return train_data, test_data

# Prepare Data
def prepare_data(train_data, test_data):
    X_train = train_data.iloc[:, 2:]
    y_train = train_data["Activity"]
    X_test = test_data.iloc[:, 2:]
    y_test = test_data["Activity"]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, y_train, X_test_scaled, y_test, scaler

# Train Model
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Evaluate Model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report




In [ ]:
if __name__ == "__main__":
    # Load and Prepare Data
    train_data, test_data = load_data(dataset_path)
    X_train, y_train, X_test, y_test, scaler = prepare_data(train_data, test_data)

    # Train Model
    model = train_model(X_train, y_train)

    # Evaluate Model
    accuracy, report = evaluate_model(model, X_test, y_test)
    print(f"Model Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{report}")

<ipython-input-5-62b56e64c903>:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  activity_labels = pd.read_csv(activity_labels_path, header=None, delim_whitespace=True, names=["Activity", "Label"])
<ipython-input-5-62b56e64c903>:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv(features_path, header=None, delim_whitespace=True, names=["Index", "FeatureName"])
<ipython-input-5-62b56e64c903>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(f"{train_path}/X_train.txt", header=None, delim_whitespace=True)
<ipython-input-5-62b56e64c903>:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use

Model Accuracy: 0.93
Classification Report:
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.91      0.90      0.90       491
          STANDING       0.91      0.92      0.91       532
           WALKING       0.89      0.97      0.93       496
WALKING_DOWNSTAIRS       0.96      0.87      0.91       420
  WALKING_UPSTAIRS       0.90      0.89      0.90       471

          accuracy                           0.93      2947
         macro avg       0.93      0.92      0.93      2947
      weighted avg       0.93      0.93      0.93      2947

